In [ ]:
def TEDeploy (base, ticker):
    ticker = ';'.join(ticker)
    url = 'https://api.tradingeconomics.com/historical/ticker/{tick}?client=guest:guest'.format(tick = ticker)
    myDF = pd.read_json(url)
    myDF['DateTime'] = pd.to_datetime(myDF['DateTime'])
    myDF = myDF.set_index('DateTime')
    myDF = myDF[['HistoricalDataSymbol', 'Value']]
    myDF = myDF.pivot_table(values='Value',
                            index=myDF.index,
                            columns='HistoricalDataSymbol',
                            aggfunc='first')  
    startDate = datetime.datetime.today() - relativedelta(years = 5)
    myDF = myDF[startDate:]
    print myDF
    baseCur =  pd.concat([base.get('historical'),base.get('points')])
    baseCur.index = pd.to_datetime(baseCur.index)
    totalDF = pd.concat([myDF,baseCur], axis = 1)
    print totalDF
    baseRow = totalDF.tail(4).iloc[[0]]
    print baseRow
    totalDFPct =  totalDF.pct_change()
    del totalDFPct.columns.name    
    testDF = totalDFPct.tail(3)
    
    corrs = totalDF.corr()[base.get('historical').columns]

    depl = []
    for t in range(len(testDF.index)):
        depl.append(corrs.T*testDF['EURUSD'][testDF.index[t]])
        depl2 = pd.concat(depl)
    depl2.index = testDF.index
    totalDFPct = totalDFPct.drop(testDF.index)
    totalDFPct = pd.concat([totalDFPct, depl2])
    print baseRow
    print totalDFPct.tail(4).iloc[[0]]
    t = baseRow.append(totalDFPct.loc[totalDFPct.tail(4).index[1]:,])
    for i in range(len(t)-1):
        t.iloc[[i+1]] = pd.DataFrame(t[i:i+2].sum()).T.values
    print t
     
    return